In [67]:
import gensim
import pandas as pd
import os
import re
import nltk

import warnings
warnings.filterwarnings('ignore')

## Labeled Words

In [61]:
rootdir = "data"
positive_words = []
negative_words = []
neutral_words = []

for directories, subdirs, files in os.walk(rootdir):
    if (os.path.split(directories)[1] == '1' or os.path.split(directories)[1] == '2' or os.path.split(directories)[1] == '3' or os.path.split(directories)[1] == '4'):
        for filename in files:
            if (filename == 'positive.txt'):
                with open(os.path.join(directories, filename)) as f:
                    for line in f:
                        positive_words.append(line.strip())
            if (filename == 'negative.txt'):
                with open(os.path.join(directories, filename)) as f:
                    for line in f:
                        negative_words.append(line.strip())
            if (filename == 'neutral.txt'):
                with open(os.path.join(directories, filename), encoding='latin-1') as f:
                    for line in f:
                        neutral_words.append(line.strip())

## Word2Vec

In [62]:
df = pd.read_csv('data/posts_data.csv')
data = df['Body']
data.dropna(inplace=True)

In [63]:
text_data = []

for line in data:
    tokens = gensim.utils.simple_preprocess(line)
    text_data.append(tokens)

In [115]:
# Stop words 
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
fil_stop = []
with open('assets/stop_words_ph.txt') as f:
    for line in f:
        fil_stop.append(line.rstrip('\n')) 

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [165]:
test_text_data = []
counter = 0
for line in data:
    tokens = gensim.utils.simple_preprocess(line)
    
    # Remove Other ADMU FW mentions
    tokens = [re.sub('\#ADMUFreedomWall\ \d+', '', sent) for sent in tokens]
    tokens =  [re.sub('\#ADMUFreedomWall\d+', '', sent) for sent in tokens]
    tokens = [word for word in tokens if word not in fil_stop]
    tokens = [word for word in tokens if word not in en_stop]
    
    test_text_data.append(tokens)
    counter += len(tokens)

print('Words in text data:', counter)
# Number of unique words
numbers_set = set(i for j in test_text_data for i in j)
print('Unique words:', len(numbers_set))
        
model = gensim.models.Word2Vec(test_text_data, size=100, window=5, 
                               min_count=10, workers=4)

model.train(test_text_data, total_examples=model.corpus_count,epochs=10)

word = 'sad'
print('Word', word)
model.wv.most_similar(word, topn=20)

Words in text data: 34400
Unique words: 5614
Word sad


[('tired', 0.99920254945755),
 ('came', 0.9990960359573364),
 ('feels', 0.9990941882133484),
 ('think', 0.999048113822937),
 ('seriously', 0.9990304112434387),
 ('yet', 0.9990196228027344),
 ('family', 0.9989954233169556),
 ('told', 0.9989808797836304),
 ('words', 0.9989672899246216),
 ('ever', 0.9989591836929321),
 ('smile', 0.9989345073699951),
 ('heart', 0.9989292621612549),
 ('one', 0.9989150166511536),
 ('come', 0.9989084601402283),
 ('others', 0.9988980293273926),
 ('leave', 0.9988976716995239),
 ('cause', 0.9988868236541748),
 ('making', 0.9988847970962524),
 ('living', 0.9988800287246704),
 ('mom', 0.9988772869110107)]

In [5]:
model = gensim.models.Word2Vec(text_data, size=100, window=10, min_count=2, workers=4)
model.train(text_data,total_examples=len(text_data),epochs=10)

(540608, 680820)

In [73]:
w1 = "cute"
model.wv.most_similar (positive=w1)

[('ayun', 0.9957599639892578),
 ('duper', 0.9957374930381775),
 ('classmate', 0.9957038164138794),
 ('markets', 0.9952459335327148),
 ('kuwentuhan', 0.995223879814148),
 ('youth', 0.9949400424957275),
 ('plzzz', 0.994774580001831),
 ('permanent', 0.9938199520111084),
 ('shorts', 0.9935779571533203),
 ('namee', 0.9935511350631714)]

# Glove

In [ ]:
import itertools
from gensim.models.word2vec import Text8Corpus
from glove import Corpus, Glove

# sentences and corpus from standard library
sentences = list(itertools.islice(Text8Corpus('text8'),None))
corpus = Corpus()

# fitting the corpus with sentences and creating Glove object
corpus.fit(sentences, window=10)
glove = Glove(no_components=100, learning_rate=0.05)

# fitting to the corpus and adding standard dictionary to the object
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)